# **Welcome to your EPAT guided mini project!**

#### Date created: 31/12/2024
#### Created by: EPAT Content Team (Led by José Carlos Gonzáles Tanaka)

<div class="alert alert-warning ">
The objective of EPAT guided projects is to give you a flavour of solving a real-world problem using Python.  

Here's what we'll do.

- We will give you the step-wise process to follow so that you understand how to analyze a problem and break it down into steps.
- We will give hints for the difficult steps. You can use them if you get stuck somewhere. This will make sure that you do not spend too much time stranded at a particular step. 
- **We urge you to search for solutions on the internet too. That's what you'll do in the outdoors and we think it is an essential part of learning to program.**
- You can explore alternate ways to solve the problem.
- Guided projects are not graded or reviewed. You need to code the solution and run it successfully to evaluate your own progress.
- We provide a model solution of the project against which you can compare your code for self-review.
- The duration of this project is ONE week. You need to complete the project within that time.


Let's begin!
</div>

# **Topic: Backtesting an Options Volatility Trading strategy with Delta Hedging**
<div class="alert alert-info ">
    <strong>Problem statement:</strong>

In the OTS module, you learned about volatility trading using options. In this notebook, we provide you with an implementation of a backtesting script using this strategy.

Specifications of the strategy:
- The volatility trading strategy presented here uses a straddle to go short whenever possible.
- The options data has a daily frequency
- We have an event-driven backtesting script iterating each day
- We use delta bands provided by the Whalley-Wilmott approximation to hedge the above strategy
- For each day, we short a straddle whenever we have a straddle signal, and we adjust the straddle delta hedge whenever the straddle delta goes outside the delta bands.
- We compute the profit and loss for each trade we open.

In this project, you will compute the following using Python:
- Determine the ATM Strike prices
- Set the parameters, the entry and exit conditions of the short straddle
- Create the event-driven backtesting loop step-by-step
- Compute trade analytics for all the trades made
- Learn two tips on how to tweak the code to improve the backtesting loop

</div>

<div class="alert alert-info ">
    <strong>Steps to perform for this project:</strong>

1. [Read the Data](#read)
2. [Calculate the ATM Strike](#atm)
3. [Strategy Parameters](#parameters)
4. [Entry Condition](#entry)
5. [Exit Conditions](#exit)
6. [Backtesting](#backtesting)
7. [Trade-Level Analytics](#analytics)
</div>

In [ ]:
# To run the code script smoothly, we suggest having the following installations
# python version 3.12 or higher
# !pip install matplotlib=3.9.2 # or higher
# !pip install numpy=1.26.4 # or higher
# !pip install pandas=2.2.3 # or higher

In [ ]:
# For data manipulation
import numpy as np
import pandas as pd

# For plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

# Import the trade analytics function
import trade_analytics_function as taf

import warnings
warnings.filterwarnings('ignore')

<a id='read'></a>
## Read the Data

- Read the following file: `options_data.bz2`
- It contains the chain data from 2016 to 2023 of the SPX EOM (End of Month) options data.

In [ ]:
# Read the options data
options_data = pd.read_pickle('options_data.bz2')
# Print the options data
options_data

Where (take note of the space before the column name):
1. **`' [STRIKE]'`**: The strike price of the options' chain
2. **`' [C_LAST]'`**: The last call price of the day
3. **`' [UNDERLYING_LAST]'`**: The last price of the underlying of the day
4. **`' [P_LAST]'`**: The last put price of the day
5. **`' [EXPIRE_DATE]'`**: The expiry date of the options' chain
6. **`' [DTE]'`**: The days to expiry of the options' chain
7. **`' [C_DELTA]'`**: The call option's delta
8. **`' [P_DELTA]'`**: The put option's delta
9. **`'IV_Rank'`**: The implied-volatility ranking computed based on the historical data
10. **`'Skew_Rank'`**: The skew-volatility ranking computed based on the historical data

In [ ]:
# Show all columns as a list
options_data.columns.tolist()

In [ ]:
# Example on how to call a column
options_data[' [STRIKE]']

<a id='atm'></a>
## Calculate the ATM Strike

Do the following:
1. Determine the appropriate interval for rounding strike prices based on the `strike_price_multiple`.
2. Calculate the ATM strike price by
    - Rounding the underlying asset's price to the nearest multiple of strike_price_multiple.
        - Divide the underlying price data from the `options_data` by the strike price multiple and round it to 0 decimals.
        - Multiple the above result by the strike price multiple
    - Add the calculated ATM strike price to the options_data DataFrame in the `atm` column.
4. Display the last 5 rows of the updated DataFrame for inspection.

In [ ]:
# To find the interval at which strike prices are rounded, set the strike multiple
strike_price_multiple = 5

# Find the strike price for the ATM
# Round the most recent price of the underlying asset to the closest multiple of the strike_price_multiple 
# to determine the ATM strike price
options_data['atm'] = ____ * (round(options_data[____] / ____))

# Show the last 5 rows of the options data
options_data.tail()

<a id='parameters'></a>
## Strategy Parameters

Both `num_days_before_expiry` and `delta_threshold` are our two parameters.

1. In order to avoid keeping the position until expiry and avoid large MTM (mark-to-market) movements as the expiration date draws near, set `num_days_before_expiry` to `3`. 

3. Set `delta_threshold` to `0.5`, which means that if the strategy's net delta rises over `0.5` or goes below `-0.5`, we shall rebalance the delta to make it come back to `0`.

These characteristics can be changed based on your trading requirements or needs.

In [ ]:
# Set the config dictionary
config = {
    'num_days_before_expiry': ____,
    'delta_threshold': ____
}

<a id='entry'></a>
## Entry Condition

The following requirements for admittance will be examined:
1. An IV rank of 50 or higher is required.

Make a column called `signal` that will hold `1` if the criteria is satisfied and `NaN` otherwise.

In [ ]:
# Verify if the IV Rank is equal to or higher than 50
options_data['signal'] = np.where(options_data[____] >= ____, ____, ____)

<a id='exit'></a>
## Exit Conditions
We will look for any one of the two exit requirements listed below:

1. The `num_days_before_expiry` is equal to or fewer than the days to expiry listed in the ` [DTE]` column of the `options_data` dataframe. Since we don't intend to take any new positions during these days, we shall keep `0` in the `signal` column.
    1. Set the `signal` to `0` if ` [DTE]` is lower than or equal to `config['num_days_before_expiry']`, otherwise, set it to the `signal` column.
    2. Set the `trade_exit_type` to `expiry_exit` if ` [DTE]` is lower than or equal to `config['num_days_before_expiry']`, otherwise, set it to the `expiry_exit` column.

3. In case you want to add them, here you have two optional exit conditions:
    1. Related to the IV rank
        1. Set the `signal` to `0` if the IV rank is less than `30`, otherwise, set it to the `signal` column.
        2. Set the `trade_exit_type` to `iv_rank_exit` if the IV rank is less than `30`, otherwise, set it to the `trade_exit_type` column.
    2. Related to the Skew rank
        1. Set the `signal` to `0` if the skew rank is either greater than or equal to `90`, or less than or equal to `10`, otherwise, set it to the `signal` column.
        2. Set the `trade_exit_type` to `iv_rank_exit` if the skew rank is either greater than or equal to `90`, or less than or equal to `10`, otherwise, set it to the `trade_exit_type` column.

In [ ]:
# Set the trade_exit_type column as NaN
options_data['trade_exit_type'] = ____

# If DTE is less than days to exit before expiry then we do not take an entry
options_data['signal'] = np.where(options_data[____] <= config[____], ____, ____)
options_data['trade_exit_type'] = np.where(options_data[____] <= config[____], ____, options_data[____])

In [ ]:
###################################################################################################
# Optional Exit conditions
###################################################################################################
# If IV rank is less than 30, set'signal' to 0
# options_data['signal'] = np.where((options_data[____] < ____), ____, options_data[____])
# options_data['trade_exit_type'] = np.where((options_data[____] < ____), ____, options_data[____])

# If skew rank is less than or equal to 10 or more than or equal to 90, set 'signal' to 0
# options_data['signal'] = np.where((options_data[____] <= ____) | (options_data[____] >= ____), ____, options_data[____])
# options_data['trade_exit_type'] = np.where((options_data[____] <= ____) | (options_data[____] >= ____), ____, options_data[____])

In [ ]:
# Set the plot
plt.figure(figsize=(15, 7))
# Plot the signal column in the options_data
options_data['signal'].plot()
# Set the plot title
plt.title('signal', fontsize=16)
# Set the plot x-axis title
plt.xlabel('Date', fontsize=15)
# Set the plot y-axis title
plt.ylabel('signal', fontsize=15)
# Show the plot
plt.show()

<a id='backtesting'></a>
## Backtesting script

Each date in the data will be looped over, the straddle will be set up when the value in the `signal` column is `1`, the trade will be updated in `trade_details`, and the delta will be rebalanced by taking a position in the underlying asset when the delta rebalancing condition is satisfied.

The strategy's option premiums, delta values, and underlying exposure for each date between the entry and exit dates are all contained in the `mtm_data` dataframe.

The following procedures will be used to backtest the straddle strategy:

**Step 1** 
1. To store round trips, trades, mark-to-market values, and adjustments, create their empty dataframes as `trade_details`, `trades`, `mtm_data`, and `adjustments`, respectively.
    - The mtm_data dataframe is to update the trade position details
    - The adjustments dataframe is for saving the upcoming new trades information regarding the delta hedging
    - The trade_details dataframe is for saving all the trades information
2. Create an `adjustaments` dataframe with the following columns: `trade_id`, `trade_entry_date`, `trade_position`, `delta_value` and `trade_entry_price`.

In [ ]:
# Set up dataframes for the trade details, the trades and the mark-to-market data
# To document all the trade's specifics
trade_details = ____
# To record individual trades
trades = ____
 # To record mark-to-market valuations
mtm_data = ____

# Initialise adjustments DataFrame
adjustments = pd.____(columns=[____, ____, ____, ____, ____])

**Step 2** 

2.1. Set up `curr_position` to monitor if we have a position at the moment. Set it as `0`.

2.2. To maintain track of the quantity of deals, set `trade_id` to `0`. 

2.3. To monitor cumulative profit and loss, set `cum_pnl` to `0`. In order to signal whether an exit condition has been met, set the `trade_close_flag` to `False`. 

2.4. Since there isn't a position at first, set the `underlying_position` to `0` and the `first_underlying_price` to `None`. 

2.5 Set the `lot_size`to 5. This means you will be shorting or buying 5 straddles per trade.

In [ ]:
# Initialize variables for current position, trade id number, cumulative PnL, and underlying position as 0
# Initial position is zero
curr_position = 0  
# Trade count starts at zero
trade_id = 0 
# Cumulative PnL starts at zero
cum_pnl = 0  
# Initialize a flag to indicate whether to exit a position to False
trade_close_flag = False  
# No initial position in the underlying asset
underlying_position = 0  
# Initial the price of the underlying asset (to be set later) to None
first_underlying_price = None
# The lot size to trade
lot_size = 5

**Step 3** 

The Whalley-Wilmott approximation offers a practical approach to hedging options when transaction costs are present. It centers around the concept of "delta bands," which define a range around the Black-Scholes delta where no trading action is taken. To hedge, we're going to use this type of band. First, let's set some variables (The variables set here and the formula applied to compute the bands are based on the OTS module. Please refer to the OTS-05 lecture for a detailed explanation of the subject).

3.1 Set the annual risk-free rate `annual_rate` to compute the Whalley-Wilmott-approximation-based delta bands.

3.2 Set the `broker_lambda` as the proportional transaction cost. You should check your broker's commission scheme to specify it here.

3.3 Set your risk-aversion ratio `kappa` as per your trade convenience.

In [ ]:
# risk-free rate
annual_rate = 0.02
# Proportional transaction cost per each straddle contract
broker_lambda = 0.05/100
# risk-aversion rate
kappa = 75.6

**Step 4**: Create a function called `update_mtm_data` that keeps track of the strategy's daily mtm_data values. 
- The function inputs are:
    1. The current `mtm_data` dataframe
    2. The `option_strategy` that will combine straddle and delta-hedging adjustments,
    3. The `trading_date` that will represent the date of corresponding trade made.

In [ ]:
# Function for calculating mark to market
def update_mtm_data(mtm_data, option_strategy, trading_date):
    # Save the trading date in the option_strategy dataframe
    option_strategy['date'] = ____
    # Concatenate the option_strategy data with the mtm_data
    mtm_data = pd.concat([____, ____])
    return mtm_data

**Step 5**: Define the function `compute_mtm_adjustment` to determine the adjustments' mark-to-market value.
- This part calculates the price difference between the current price of the underlying asset `underlying_price` and the price at which the trade was initially entered `row['trade_entry_price']`.
- Multiply the above with the `row['trade_position']`. This represents the direction of the trade (e.g., 1 for long, -1 for short).

In [ ]:
# Function to calculate the mark-to-market value of adjustments
def compute_mtm_adjustment(row, underlying_price):
    # Subtract the row's entry price to the underlying price and multipliy the result by the row trade position
    return (____ - row[____]) * row[____]

**Step 6**: To obtain the premium for the straddle's two legs (CE and PE), define the function `get_option_premium`. It accepts two inputs: `options_strategy` and `options_data`.
- If the `type_of_asset` is a call option, then return the call last price
- If the `type_of_asset` is a put option, then return the put last price

In [ ]:
# Function for fetching premium
def get_option_premium(options_strategy, options_data):

    # Get the premium for call option
    if options_strategy[____] == "CE":
        # Return the options_data call price last value
        return options_data[____]

    # Get the premium for put option
    elif options_strategy[____] == "PE":
        # Return the options_data put price last value
        return options_data[____]

**Step 7**

- To obtain the delta for the straddle's two legs (CE and PE), define the method `get_delta_value`. It accepts two inputs: `options_strategy` and `options_data`.
    - If the `type_of_asset` is a call option, then return the call delta value
    - If the `type_of_asset` is a put option, then return the put delta value
- To obtain the gamma for the straddle's two legs (CE and PE), define the method `get_gamma_value`. It accepts two inputs: `options_strategy` and `options_data`.
    - If the `type_of_asset` is a call option, then return the call gamma value
    - If the `type_of_asset` is a put option, then return the put gamma value

In [ ]:
# Function for fetching delta
def get_delta_value(options_strategy, options_data):

    # Get the delta for call option
    if options_strategy['type_of_asset'] == "CE":
        # Return the options_data call delta last value
        return options_data[' [C_DELTA]']

    # Get the delta for put option
    elif options_strategy['type_of_asset'] == "PE":
        # Return the options_data put delta last value
        return options_data[' [P_DELTA]']

# Function for fetching gamma
def get_gamma_value(options_strategy, options_data):

    # Get the delta for call option
    if options_strategy['type_of_asset'] == "CE":
        # Return the options_data call gamma last value
        return options_data[' [C_GAMMA]']

    # Get the delta for put option
    elif options_strategy['type_of_asset'] == "PE":
        # Return the options_data put gamma last value
        return options_data[' [P_GAMMA]']

**Step 8**: To configure the straddle approach, define a function called `setup_straddle`. It accepts two inputs: `options_data` and `direction` (long/short). Inside the function, do the following:
- Create an empty dataframe called `straddle`.
- Create the column `type_of_asset` for the `straddle` dataframe where the values will be `['CE', 'PE']`.
- Set the `strike_price` of the straddle as the first index of the options_data `atm` column.
- Set the `trade_position` to `-1` if the `direction` input is `short`.
- Set the `trade position` to `1` in case the `direction` input is `long`.
- Set the `premium` values for both call and put legs using a `lambda` function together with the `get_option_premium` function by setting its inputs as
    - The `straddle` dataframe, and
    - The `options_data` dataframe
- Set the `delta_value` using a `lambda` function together with the `get_delta_value`function by setting its input as
    - The `straddle` dataframe, and
    - The `options_data` dataframe
- Set the `gamma_value` using a `lambda` function together with the `get_gamma_value`function by setting its input as
    - The `straddle` dataframe, and
    - The `options_data` dataframe

In [ ]:
# Function for setting up a straddle
def setup_straddle(options_data, direction='short'):

    # Create a dataframe to store the straddle
    straddle = ____

    # Save in the type_of_asset column both the CE and PE legs of the straddle
    straddle[____] = [____, ____]

    # Set the strike_price column of the straddle dataframe to at-the-money price
    straddle[____] = options_data[____].iloc[0]

    # Set the trade_position to -1 for both CE and PE legs in case of a short straddle
    straddle[____] = ____

    # If the direction is "long"
    if direction == 'long':
        # Set the trade_position to 1 for both CE and PE legs in case of a long straddle
        straddle[____] = ____

    # Set the premium column to the get_option_premium return value using 
    # each straddle's row info and options_data as the function's inputs
    straddle[____] = straddle.apply(lambda r: get_option_premium(____, ____), axis=1)

    # Set the delta_value column to the get_delta_value return value using 
    # each straddle's row info and options_data as the function's inputs
    straddle[____] = straddle.apply(lambda r: get_delta_value(____, ____), axis=1)
    
    # Set the gamma_value column to the get_gamma_value return value using 
    # each straddle's row info and options_data as the function's inputs
    straddle[____] = straddle.apply(lambda r: get_gamma_value(____, ____), axis=1)

    return straddle

**Step 9**: Configure the backtesting `start_date`.
- Set the `start_date` as the first options_data index.

In [ ]:
# Set the start date for backtesting as the options data's first index value
start_date = options_data.index[____]

For the dates in the backtest period, iteratively complete the following steps.

**Step 10**: If there is an open position on a specified date and the exit conditions are satisfied, we exit the trade. We look for the delta balancing condition if the exit conditions are not satisfied. To make the portfolio delta neutral, we trade the underlying if the net delta of the positions exceeds an absolute value of `0.5`. 

**Step 11**: We will set up the straddle if there isn't an available position on a certain date and the entrance requirement is satisfied.
The for-loop goes like the following:
1. Subset the current day data and its underlying price
2. Compute the current day trades information for an existing open position.
    1. Compute the trades information for the volatility position
    2. Compute the current day trades information for the delta hedging position in case we had opened it.
    3. Check whether in the current day we have an exit signal or not
        1. If we have an exit signal, we close the position and obtain the trades information of the position
            1. Computing the trades information for the volatility position to close it
            2. Compute the current day trades information for the delta hedging position to close it in case we have it.
            3. Update the whole strategy trades information for the current day
        2. If we don't have an exit signal, update the trades information of the volatility position for the current day
            1. Updating the delta bands
            2. Update the volatility position
            3. Update the delta hedging position by checking if it breaches the delta threshold
3. Compute the trades information in case we have a new signal

In [ ]:
for i in options_data.loc[start_date:].index.unique():

    ################################################################################################################
    # Section 1: Computing the trade information in case we have an open position
    ################################################################################################################
    # Get the current-day options data
    current_day_data = options_data.loc[____,:]  
    
    # Get the underlying price for the current day
    underlying_price = current_day_data[____].iloc[0]

    ################################################################################################################
    # Section 2: Computing the trades information in case we have an open position
    ################################################################################################################
    # If we have an open position from the previous date
    if curr_position == 1:  
        ################################################################################################################
        # Section A: Computing the trades information for the volatility position
        ################################################################################################################
        
        # Subset the current_day_data by selecting only the data that belongs to the setup_strike
        current_strike_data = current_day_data[current_day_data[____] == setup_strike]
        # Set the premium column to the get_option_premium return value using 
        # each straddle's row info and current_strike_data as the function's inputs        straddle[____] = straddle.apply(lambda r: get_option_premium(____, ____), axis=____)  
        # Update the delta of the straddle
        straddle[____] = straddle.apply(lambda r: get_delta_value(____, ____), axis=____) 
        # Set the delta_value column to the get_option_premium return value using 
        # each straddle's row info and current_strike_data as the function's inputs
        straddle[____] = straddle.apply(lambda r: get_gamma_value(____, ____), axis=1)  
        # Set the gamma_value column to the get_option_premium return value using 
        # each straddle's row info and current_strike_data as the function's inputs
        net_premium = (straddle.____ * straddle.____).sum()

        # Update the mark-to-market dataframe with the straddle dataframe and the options_data index "i" value
        mtm_data = update_mtm_data(mtm_data, ____, ____)

        ################################################################################################################
        # Section B: Computing the trades information for the delta hedging position if we had opened it
        ################################################################################################################
        # Include adjustments in the mark-to-market
        if not adjustments.empty:
            # Compute the mark-to-market values of the delta-hedging adjustment and save it in the adjustments "mtm" column
            # Use the each adjustments' row information together with the underlying_price variable as inputs for 
            # the compute_mtm_adjustment function
            adjustments['mtm'] = adjustments.apply(lambda ____: compute_mtm_adjustment(____, ____), axis=1)
            
            # Add the adjustments data into the mark-to-market dataframe
            for index, row in adjustments.iterrows():
                # Fill the adjustment_entry dictionary with the adjustments row information
                adjustment_entry = {
                    # Set the type_of_asset as the same row information from the adjustment data row
                    'type_of_asset': row[____],
                    # Set the strike price as NaN
                    'strike_price': ____,
                    # Set the trade position as the same row information from the adjustment data row
                    'trade_position': row[____],
                    # Set the premium as the same row information from the adjustment data row
                    'premium': row[____],
                    # Set the delta value as the same row information from the adjustment data row
                    'delta_value': row[____],
                    # Set the gamma value as the same row information from the adjustment data row
                    'gamma_value': row[____],
                    # Set the date as the options_data index "i"
                    'date': i
                }

                # Fill at the bottom of the mark-to-market dataframe the adjustment_entry data
                mtm_data.loc[len(mtm_data),____.keys()] = ____.values()

            # Drop the mtm column from the adjustaments dataframe
            adjustments.drop(____, axis=1, inplace=True)

        # Check if any exit conditions are met (Exit signal is 0)
        if current_day_data['signal'].iloc[0] == 0: 
            # Set trade_exit_type variable to the the current_day_data's trade_exit_type first value
            trade_exit_type = current_day_data[____].iloc[0]
            # Set the trade_close_flag to True
            trade_close_flag = ____

        ################################################################################################################
        # Section C: Check whether if in the current day we have an exit signal or not
        ################################################################################################################
        ################################################################################################################
        # Section a: If we have an exit signal, we close the position and obtain the trade information of the position
        ################################################################################################################
        if trade_close_flag:
            ################################################################################################################
            # Section i: Computing the trade information for the volatility position to close it
            ################################################################################################################
            # Record the exit date as the options_data index "i" value
            trades['trade_exit_date'] = ____  
            # Record the exit type as the trade_exit_type variable
            trades['trade_exit_type'] = ____  
            # Record the trade exit price as the current straddle premium value
            trades['trade_exit_price'] = straddle.____  

            # Ensure there are adjustments for the current trade
            if not adjustments.empty and trade_id in adjustments['trade_id'].values:
                # Set the trade_exit_date corresponding to the trade_id row to the options_data index "i" value
                adjustments.loc[adjustments[____]== trade_id, ____] = ____
                # Set the trade_exit_price corresponding to the trade_id row to the underlying_price
                adjustments.loc[adjustments[____] == ____, ____] = ____

            # Calculate net premium again for the straddle position as we close the position
            # Round the sum of the  values obtained from multiplication between the straddle's trade_position and premium to 1
            net_premium = round((straddle.____ * straddle.____).____(), ____)
            
            # Calculate entry net premium of the straddle position
            # Compute the sum of the values obtained from multiplication between the trades's trade_position and trade_entry_price
            entry_net_premium = (trades.____ * trades.____).sum()
            # Calculate the PnL for the trade as the difference between the trades' trade_entry_price and trade_exit_price
            # Save it in the "pnl" column
            trades['pnl'] = trades[____] - trades[____] 
            # Calculate the total PnL for the trade and round the value to 1 decimal
            # Round the difference between the net_premium and the entry_net_premium to 1
            trade_pnl = round(____ - ____, 1)

            ################################################################################################################
            # Section ii: Computing the trades information for delta hedging to close the underlying trade
            ################################################################################################################
            if first_underlying_price is not None and adjustments.empty:
                # Create a delta_trade dataframe
                delta_trade = pd.DataFrame({
                    # Save the type of asset as "underlying"
                    'type_of_asset': ____,
                    # There is no strike price so just type NA
                    'strike_price': ____,
                    # Save the underlying price as the 2-decimal rounded underlying_position
                    'trade_position': round(____, ____),
                    # Save the delta value as the 2-decimal rounded underlying_position
                    'delta_value': round(____, ____),
                    # Record initial date as the initial_underlying_date
                    'trade_entry_date': ____,  
                    # Record the trade day as the options_data index "i" value
                    'trade_exit_date': [____],
                    # Record exit type as "delta_hedge"
                    'trade_exit_type': ____,  
                    # Record the initial price as the first_underlying_price
                    'trade_entry_price': ____,  
                    # Record the exit price as the underlying price
                    'trade_exit_price': ____,
                    # Record underlying PnL
                    # Subtract the first_underlying_price to the underlying_price and multiply the result by the underlying_position
                    'pnl': (____ - ____) * ____
                })
                # Concatenate the new trade details
                trades = pd.concat([____, ____])
            
            ################################################################################################################
            # Section iii: Updating the whole strategy trades information for the current day
            ################################################################################################################
            # Record the "trades" information into the trade details dataframe
            trade_details = pd.concat([____, ____])

            # Append adjustments to the trade details
            if not adjustments.empty and trade_id in adjustments['trade_id'].values:
                # Save the pnl of today's trade by multiplying the adjustments' trade_position to 
                # the difference between the adjustments' exit and entry price
                adjustments['pnl'] = (adjustments['trade_exit_price'] - 
                                      adjustments[____]) * adjustments[____]
                # Concatenate the adjustment trade details corresponding to the trade_id with the trade_details dataframe
                trade_details = pd.concat([____, adjustments[adjustments['trade_id'] == ____]])
                # Adjust the PnL to sum the adjustments PnL
                # Sum the column "pnl" values of the adjustments dataframe and sum the result to the trade_pnl
                trade_pnl += adjustments.____.____()

            # Update the cumulative PnL by summing the trade_pnl
            cum_pnl += ____  
            # Round the cumulative PnL to 2 decimals
            cum_pnl = round(____, ____)  
            print(f"- Trade No: {trade_id} | Exit Type: {trade_exit_type} | Date: {i} | Premium: {net_premium}")
            print(f"\t Delta: {round(net_delta, 2)} | Trade PnL: {round(trade_pnl, 2)} | Cum PnL: {round(cum_pnl, 2)}")
            # Reset the current position to 0
            curr_position = ____  
            # Reset initial underlying price to None
            first_underlying_price = ____  
            # Reset initial underlying date to None
            initial_underlying_date = ____  
            # Reset the underlying position to 0
            underlying_position = ____  
            # Reset exit flag to False
            trade_close_flag = ____  

            # Clear the adjustments data from the information related to the current trade id
            adjustments = adjustments[adjustments[____] != ____]

        ################################################################################################################
        # Section b: If we don't have an exit signal, update the trade information of the volatility position
        ################################################################################################################
        else:  # If no exit conditions are met  
            
            ################################################################################################################
            # Section i: Updating the delta bands
            ################################################################################################################
            # Set the previous-day options' days to expiry as prev_dte
            ____ = float(previous_day_data[____].iloc[0])
            # Set the previous last underlying price as prev_last
            ____ = float(previous_day_data[____].iloc[0])
            # Compute the straddle position delta value
            prev_straddle_delta = (prev_straddle.____ * prev_straddle.____).sum()
            # Compute the straddle position gamma value
            prev_straddle_gamma = (prev_straddle.____ * prev_straddle.____.astype(float)).sum()
        
            # Compute the Whalley-Wilmott approximation-based delta bands
            # The formula to compute the bands have 2 parts. The first one is the delta value and the second part is a math computation
            # of different variables. Please check the OTS-05 lecture to find out how to compute the bands
            # Compute the second part of the bands' equation:
            second_part = ((3/2)*(np.exp(-annual_rate*(prev_dte/____))*broker_lambda*prev_last*prev_straddle_gamma**2)/____)**(1/____)
        
            # If the straddle position is long
            if straddle.trade_position.iloc[0] == 1:
                # Compute the upper band for the long straddle
                delta_upper_band = prev_straddle_delta + ____
                # Compute the lower band for the long straddle
                delta_lower_band = prev_straddle_delta - ____
            # If the straddle position is short
            elif straddle.trade_position.iloc[0] == -1:
                # Compute the lower band for the short straddle
                delta_upper_band = prev_straddle_delta + ____
                # Compute the upper band for the short straddle
                delta_lower_band = prev_straddle_delta - ____

            # Compute the average delta based on the above two bands to adjust the delta position if needed
            delta_band_avg_value = (____+____)/2
            
            ################################################################################################################
            # Section ii: Updating the volatility position
            ################################################################################################################
            # Calculate the net delta including the underlying position
            try:
                # Compute the straddle position delta value
                straddle_delta = (straddle.____ * straddle.____).sum()
                # Compute the straddle position gamma value
                straddle_gamma = (straddle.____ * straddle.____.astype(float)).sum()
                # Save the underlying_position as the underlying_delta
                underlying_delta = ____
                # Save the net delta of the straddle strategy by summing the underlying position to the straddle_delta
                net_delta = ____ + ____
                
            # In case we don't have enough information
            except KeyError:
                print(f"- Data missing for the required strike prices on {i}, Not adding to trade logs.")
                # Reset position if data is missing
                curr_position = 0  
                continue

            ################################################################################################################
            # Section iii: Updating the delta hedging position by checking if it breaches the delta threshold
            ################################################################################################################
            # Check if the net delta exceeds the threshold
            # If we are long the straddle
            if straddle.trade_position.iloc[0] == ____:
                # Check if the straddle net delta is out of the bands
                delta_band_breaching = ((net_delta > delta_upper_band) or (net_delta < delta_lower_band))
            # If we have shorted the straddle
            elif straddle.trade_position.iloc[0] == ____:
                # Check if the straddle net delta is out of the bands
                delta_band_breaching = ((net_delta < delta_lower_band) or (net_delta > delta_upper_band))

            # If the straddle net delta is out of the bands
            if delta_band_breaching:
                # Compute the delta adjustment needed
                # Subtract the delta_band_avg_value to the net_delta to set the delta adjustment
                adjustment = -(____ - ____)
                # Adjust the underlying position by summing the adjustment to the underlying_position 
                underlying_position += ____ 
                # Set the underlying price as the first underlying price
                first_underlying_price = ____
                
                # Create the delta trade dataframe
                delta_trade = pd.DataFrame({
                    # Save the trade id
                    'trade_id': trade_id,
                    # Set the type of asset as "underlying"
                    'type_of_asset': ____,
                    # Set the strike price of the underlying as "NA"
                    'strike_price': ____,
                    # Set the date of the adjustment
                    'trade_entry_date': [____],
                    # Set the trade position of the hedging as the delta adjustment
                    'trade_position': round(____, 2),
                    # Set the delta value
                    'delta_value': round(adjustment, 2),
                    # Set the gamma value
                    'gamma_value': 0.0,
                    # Set the entry price (price at adjustment) as the first_underlying_price
                    'trade_entry_price': ____,
                    # Set the exit type of the hedging as "delta_hedge" 
                    'trade_exit_type': "____"
                })

                # Add details to adjustments
                # Concatenate the adjustments to the delta_trade dataframe
                adjustments = pd.concat([____, ____], ignore_index=True)
                
                print(f"- Delta hedge | date: {i} | straddle delta: {round(straddle_delta, 2)} | underlying: {round(underlying_delta, 2)}")
                print(f"\t net delta: {round(net_delta, 2)} | adjustment: {round(adjustment, 2)}")

            prev_straddle = straddle.copy()

    ################################################################################################################
    # Section 3: Computing the trade information in case we have a new signal
    ################################################################################################################

    # If there is no current position and the entry signal is 1
    if curr_position == 0 and current_day_data['signal'].iloc[0] == 1:
        # Reset trade PnL to 0
        trade_pnl = ____ 
        
        # Subset the current_day_data as per the data that belongs to the ATM strike price
        current_strike_data = current_day_data[current_day_data[____] == current_day_data[____]]

        # Setup a new short straddle
        straddle = setup_straddle(____, direction="short")
        # Set the strike price as the "atm" first value
        setup_strike = current_day_data[____].iloc[____]  

        # Copy the straddle dataframe details to the trades dataframe
        trades = straddle.copy()  
        # Save the trade entry date as the options_data index "i" value
        trades['trade_entry_date'] = ____
        
        # Rename the trades entry_price column as "premium"
        trades.rename(columns={____: ____}, inplace=____)

        # Calculate the net premium of the straddle 
        # Round the sum of the values obtained from multiplication between the straddle's trade_position and premium to 2
        net_premium = round((straddle.____ * straddle.____).____(), ____)

        try:
            # Compute the net delta
            # Round the sum of the values obtained from multiplication between the straddle's trade_position and delta_value to 2
            net_delta = round((straddle.____ * straddle.____).sum(), 2)
        except KeyError:
            print(f"- Data missing for the required strike prices on {i}, Not adding to trade logs.")
            # Reset the current position to 0 if data is missing
            curr_position = ____  
            continue

        # Set the current position to 1
        curr_position = ____  
        
        # Add the straddle details to the mark-to-market data
        # Use the mtm_data, the straddle dataframe and the options_data index "i" value as inputs for the update_mtm_data
        mtm_data = update_mtm_data(____, ____, ____)  

        # Increment trade id number
        trade_id += ____  

        # Save the trade id in the trades dataframe
        trades['trade_id'] = ____
        
        print("-" * 30)
        print(f"- Trade No: {trade_id} | Entry Date: {i} | Premium: {net_premium} | Initial Delta: {round(net_delta, 2)} | Trade PnL: {trade_pnl} | Cum PnL: {cum_pnl}")     

    # Save the current day data as previous day's to compute the delta bands
    previous_day_data = current_day_data.copy()

<a id='analytics'></a>
## Trade-Level Analytics

Let's print the `trade_details` dataframe

In [ ]:
# Set the trade_id as the trade_details index
trade_details = trade_details.set_index([____])
# Show the first 10 trades
trade_details.tail(10)

Let's print the mark-to-market (`mtm`)information.

In [ ]:
# Show the mark-to-market details
mtm_data.tail(10)

Let's print the trade-level analytics. The function needs 2 inputs:
1. The trade_details dataframe, and
2. The lot size to be used while trading. 1 means investing in a single straddle strategy, 2 means investing in 2 straddles, etc.

In [ ]:
# Trade Level Analytics
# Assume lot size as 5
analytics = taf.trade_level_analytics(____, ____)
analytics

## Notes

- The prior analysis did not consider transaction costs. To determine the true impact of delta hedging, we must examine the final PnL after accounting for the strategy's transaction costs.
- We have set a short-only straddle strategy, try tweaking the code by creating a long-short strategy.

## Conclusion

In this notebook, we create a backtesting script of a volatility-trading strategy and delta hedging. We encourage you to tweak the code to make a sensitivity analysis to find an optimized delta threshold, IV rank and Skew rank.